In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

%matplotlib inline
# Any results you write to the current directory are saved as output.

## Data Loading

In [ ]:
TRAINPATH = os.path.join("..", "input", "train.csv")
TESTPATH = os.path.join("..", "input", "test.csv")

In [ ]:
df = pd.read_csv(TRAINPATH, index_col=0)
df.head()

In [ ]:
df_test = pd.read_csv(TESTPATH)
df_test.head()

In [ ]:
df.info()

le data set d'entrainement comporte 10 colonnes
l id en nombre entier
le decompte des passagers et le temps de trajet également
longitude et latitude de depart et d'arrivée en nombre flotant


## valeurs manquantes

In [ ]:
df.duplicated().sum()

In [ ]:
## suppression des valeurs récurrentes

df = df.drop_duplicates()
df.duplicated().sum()



In [ ]:
# valeurs nulles 

df.isna().sum()

### Outliers detection

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
df.boxplot(['trip_duration'], fontsize=12)
fig.suptitle('Visualisation des outliers', fontsize=20)

In [ ]:
CAT_VARS = ['store_and_fwd_flag']

In [ ]:
for col in CAT_VARS:
    df[col] = df[col].astype('category').cat.codes
df.head()

In [ ]:
for col in CAT_VARS:
    df_test[col] = df_test[col].astype('category').cat.codes
df_test.head()

## Features engineering 

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
coords = np.vstack((df[['pickup_latitude', 'pickup_longitude']].values,
                    df[['dropoff_latitude', 'dropoff_longitude']].values,
                    df_test[['pickup_latitude', 'pickup_longitude']].values,
                    df_test[['dropoff_latitude', 'dropoff_longitude']].values))

pca = PCA().fit(coords)

# Train
df['pickup_pca0'] = pca.transform(df[['pickup_latitude', 'pickup_longitude']])[:, 0]
df['pickup_pca1'] = pca.transform(df[['pickup_latitude', 'pickup_longitude']])[:, 1]
df['dropoff_pca0'] = pca.transform(df[['dropoff_latitude', 'dropoff_longitude']])[:, 0]
df['dropoff_pca1'] = pca.transform(df[['dropoff_latitude', 'dropoff_longitude']])[:, 1]

# Test
df_test['pickup_pca0'] = pca.transform(df_test[['pickup_latitude', 'pickup_longitude']])[:, 0]
df_test['pickup_pca1'] = pca.transform(df_test[['pickup_latitude', 'pickup_longitude']])[:, 1]
df_test['dropoff_pca0'] = pca.transform(df_test[['dropoff_latitude', 'dropoff_longitude']])[:, 0]
df_test['dropoff_pca1'] = pca.transform(df_test[['dropoff_latitude', 'dropoff_longitude']])[:, 1]

In [ ]:
df.head()

Creation of month, day & hour features

In [ ]:
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
df['dropoff_datetime'] = pd.to_datetime(df['dropoff_datetime'])
df_test['pickup_datetime'] = pd.to_datetime(df_test['pickup_datetime'])

In [ ]:
df['hour'] = df.pickup_datetime.dt.hour
df['day'] = df.pickup_datetime.dt.dayofweek
df['month'] = df.pickup_datetime.dt.month
df_test['hour'] = df_test.pickup_datetime.dt.hour
df_test['day'] = df_test.pickup_datetime.dt.dayofweek
df_test['month'] = df_test.pickup_datetime.dt.month

Distance feature

In [ ]:
df['distance2'] = np.sqrt((df['pickup_pca0']-df['dropoff_pca0'])**2
                        + (df['pickup_pca1']-df['dropoff_pca1'])**2)
df_test['distance2'] = np.sqrt((df_test['pickup_pca0']-df_test['dropoff_pca0'])**2
                        + (df_test['pickup_pca1']-df_test['dropoff_pca1'])**2)

Log transformation

In [ ]:
df['log_trip_duration'] = np.log(df['trip_duration'])


In [ ]:
df.head()

In [ ]:
df_test.head()

## Features selection 

In [ ]:
NUM_VARS = ['pickup_pca0', 'pickup_pca1', 'dropoff_pca0', 'dropoff_pca1', 'month', 'hour', 'day', 'distance2']
TARGET = 'log_trip_duration'

In [ ]:
num_features = NUM_VARS


In [ ]:
X_train = df.loc[:, num_features]
y_train = df[TARGET]
X_test = df_test.loc[:, num_features]
X_train.shape, y_train.shape, X_test.shape

### training model

In [ ]:
from sklearn.ensemble import RandomForestRegressor
# une analyse par regression semble plus pertinente sur ce modele


In [ ]:
training = RandomForestRegressor(n_estimators=19, min_samples_leaf=10, min_samples_split=15, max_features='auto', max_depth=80, bootstrap=True)
training.fit(X_train, y_train)

In [ ]:
from sklearn.model_selection import cross_val_score


In [ ]:
cv_scores = cross_val_score(training, X_train, y_train, cv=5, scoring='neg_mean_squared_log_error')

In [ ]:
cv_scores

In [ ]:
for i in range(len(cv_scores)):
    cv_scores[i] = np.sqrt(abs(cv_scores[i]))
cv_scores

## predictions

In [ ]:
y_test_pred = training.predict(X_test)
y_test_pred[:5]

In [ ]:
predection = pd.DataFrame({'id': df_test.id, 'trip_duration': np.exp(y_test_pred)})
# you could use any filename. We choose submission here
predection.to_csv('submission.csv', index=False)

In [ ]:
predection.head()